In [49]:
import os

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='1,2,3,4,5,6,7'

In [50]:
from datasets import load_dataset
  
dataset = load_dataset("gyr66/privacy_detection")

In [51]:
dataset = dataset["train"].train_test_split(train_size=0.8, seed=42)
dataset["validation"] = dataset.pop("test")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2012
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 503
    })
})

In [52]:
from transformers import AutoTokenizer

# model_checkpoint = "hfl/chinese-roberta-wwm-ext"
model_checkpoint = "gyr66/RoBERTa-finetuned-privacy-detection"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [53]:
def align_labels_with_tokens(labels, word_ids):
    return [-100 if word_id is None else labels[word_id] for word_id in word_ids]

In [54]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, max_length=512
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    assert all([len(labels) == len(input_ids) for labels, input_ids in zip(tokenized_inputs["labels"], tokenized_inputs["input_ids"])])
    return tokenized_inputs

In [55]:
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/2012 [00:00<?, ? examples/s]

Map:   0%|          | 0/503 [00:00<?, ? examples/s]

In [56]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [57]:
import evaluate

metric = evaluate.load("seqeval")

In [58]:
ner_feature = dataset["train"].features["ner_tags"]
label_names = ner_feature.feature.names
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [59]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [60]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

In [61]:
from transformers import TrainingArguments

args = TrainingArguments(
    "RoBERTa-finetuned-privacy-detection",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    per_device_train_batch_size=8,
    logging_strategy="epoch",
    dataloader_num_workers=16,
)

In [62]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/home/yirguo/projects/privacy_detection/RoBERTa-finetuned-privacy-detection is already a clone of https://huggingface.co/gyr66/RoBERTa-finetuned-privacy-detection. Make sure you pull the latest changes with `repo.git_pull()`.
/mnt/ds3lab-scratch/yirguo/miniconda3/envs/ADMM/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/mnt/ds3lab-scratch/yirguo/miniconda3/envs/ADMM/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; w

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.202700,0.348493,0.591281,0.727341,0.652292,0.902954
2,0.165200,0.353434,0.615336,0.731370,0.668354,0.905250
3,0.143000,0.353391,0.616885,0.724824,0.666512,0.906119


/mnt/ds3lab-scratch/yirguo/miniconda3/envs/ADMM/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/ds3lab-scratch/yirguo/miniconda3/envs/ADMM/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/mnt/ds3lab-scratch/yirguo/miniconda3/envs/ADMM/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/ds3lab-scratch/y

TrainOutput(global_step=108, training_loss=0.1703277517248083, metrics={'train_runtime': 258.1591, 'train_samples_per_second': 23.381, 'train_steps_per_second': 0.418, 'total_flos': 1489544768270328.0, 'train_loss': 0.1703277517248083, 'epoch': 3.0})

In [68]:
trainer.push_to_hub(commit_message="Training complete", language="zh", dataset_tags="gyr66/privacy_detection")

To https://huggingface.co/gyr66/RoBERTa-finetuned-privacy-detection
   1e68d77..e4a7c04  main -> main

